In [1]:
#!/usr/bin/env python3
import numpy as np
import tf.transformations as tf
from math import *
import cmath
from geometry_msgs.msg import Pose, Quaternion
from math import radians

from visual_kinematics.RobotSerial import *
from visual_kinematics.RobotTrajectory import *
import numpy as np
from math import pi

In [2]:

#-------------------------------------------------------------------------------------------------
#D-H Parameter untuk Visual Kinematics
#[d,a, alpha, theta]
"""dh_params = np.array([  [0.1519,   0.,       pi/2,   0.],
                        [0.,      -0.24365,  0.,     0.],
                        [0.,      -0.21325,  0.,     0.],
                        [0.11235,  0.,       pi/2,   0.],
                        [0.08535,  0.,       -pi/2,  0.],
                        [0.0819,   0.,       0.,     0.]])
robot = RobotSerial(dh_params)"""

# Inisialisasi Parameter Denavit-Hartenberg (Teoritikal) (Hitung Manual)
# d (unit: meter)
d1 = 0.1519 
d2 = d3 = 0
d4 = 0.11235
d5 = 0.08535
d6 = 0.0819

# a (unit: mm)
a1 = a4 = a5 = a6 = 0
a2 = -0.24365
a3 = -0.21325

# List type of D-H parameter
d = np.array([d1, d2, d3, d4, d5, d6]) # unit: mm
a = np.array([a1, a2, a3, a4, a5, a6]) # unit: mm
alpha = np.array([pi/2, 0, 0, pi/2, -pi/2, 0]) # unit: radian

In [3]:
# Convert dari UR Format ke ROS Pose Format

def ur2ros(ur_pose):
    """Convert UR format pose to ROS Pose format."""
    ros_pose = Pose()

    # ROS position
    ros_pose.position.x = ur_pose[0]
    ros_pose.position.y = ur_pose[1]
    ros_pose.position.z = ur_pose[2]

    # Orientation in UR format
    rx, ry, rz = ur_pose[3], ur_pose[4], ur_pose[5]

    # Convert orientation from Euler angles to quaternion
    cy = cos(rz * 0.5)
    sy = sin(rz * 0.5)
    cr = cos(rx * 0.5)
    sr = sin(rx * 0.5)
    cp = cos(ry * 0.5)
    sp = sin(ry * 0.5)

    ros_pose.orientation.x = cy * cr * sp - sy * sr * cp
    ros_pose.orientation.y = cy * sr * cp + sy * cr * sp
    ros_pose.orientation.z = sy * cr * cp - cy * sr * sp
    ros_pose.orientation.w = cy * cr * cp + sy * sr * sp
    
    return ros_pose

In [4]:
#-------------------------------------------------------------------------------------------------
# Transformasi Pose dari ROS Pose format ke np.array format

def ros2np(ros_pose):
    # orientation
    np_pose = tf.quaternion_matrix([ros_pose.orientation.x, ros_pose.orientation.y,
                                    ros_pose.orientation.z, ros_pose.orientation.w])
    
    # position
    np_pose[0][3] = ros_pose.position.x
    np_pose[1][3] = ros_pose.position.y
    np_pose[2][3] = ros_pose.position.z

    return np_pose


In [5]:
#-------------------------------------------------------------------------------------------------
# Transformasi Pose dari np.array format ke ROS Pose format

def np2ros(np_pose):
    # ROS pose
    ros_pose = Pose()

    # ROS position
    ros_pose.position.x = np_pose[0, 3]
    ros_pose.position.y = np_pose[1, 3]
    ros_pose.position.z = np_pose[2, 3]

    # ROS orientation 
    np_q = tf.quaternion_from_matrix(np_pose)
    ros_pose.orientation.x = np_q[0]
    ros_pose.orientation.y = np_q[1]
    ros_pose.orientation.z = np_q[2]
    ros_pose.orientation.w = np_q[3]

    return ros_pose

In [6]:
#Fungsi Matriks Transformasi Homogen
def HTM(i, theta):
    Rot_z = np.matrix(np.identity(4))
    Rot_z[0, 0] = Rot_z[1, 1] = cos(theta[i])
    Rot_z[0, 1] = -sin(theta[i])
    Rot_z[1, 0] = sin(theta[i])

    Trans_z = np.matrix(np.identity(4))
    Trans_z[2, 3] = d[i]

    Trans_x = np.matrix(np.identity(4))
    Trans_x[0, 3] = a[i]

    Rot_x = np.matrix(np.identity(4))
    Rot_x[1, 1] = Rot_x[2, 2] = cos(alpha[i])
    Rot_x[1, 2] = -sin(alpha[i])
    Rot_x[2, 1] = sin(alpha[i])

    A_i = Rot_z * Trans_z * Trans_x * Rot_x	    
    return A_i

In [7]:
# Forward Kinematics
def fwd_kin(theta, i_unit='r', o_unit='n'):
    T_06 = np.matrix(np.identity(4))

    if i_unit == 'd':
        theta = [radians(i) for i in theta]
    
    for i in range(6):
        T_06 *= HTM(i, theta)

    if o_unit == 'n':
        return T_06
    elif o_unit == 'p':
        return np2ros(T_06)
    else:
        print("Forward Kinematics Result:")
        print(T_06)  

In [8]:
joint_values = [radians(5), radians(-57), radians(45), radians(15), radians(5), radians(5)]
#Hasil Forward Kinematics Hitung Manual
result = fwd_kin(joint_values, i_unit='r', o_unit='p')
result_htm = fwd_kin(joint_values, i_unit='r', o_unit='n')   # Menggunakan 'p' untuk output format ROS Pose

print("ROS Pose Forward Kinematics (Hitung Manual):")
print(result)
print()
print('**************** INPUT INVERSE KINEMATICS')
print('Hasil Forward Kinematics (Matriks Transformasi Homogen) ')
print(result_htm)

ROS Pose Forward Kinematics (Hitung Manual):
position: 
  x: -0.3257409914003134
  y: -0.223177802741218
  z: 0.31497264420402227
orientation: 
  x: 0.7069990853988242
  y: -0.04932527561613232
  z: 0.049184538196814814
  w: 0.7037756685428851

**************** INPUT INVERSE KINEMATICS
Hasil Forward Kinematics (Matriks Transformasi Homogen) 
[[ 9.90295797e-01 -1.38975612e-01  1.18989396e-04 -3.25740991e-01]
 [-5.16086992e-04 -4.53365110e-03 -9.99989590e-01 -2.23177803e-01]
 [ 1.38974705e-01  9.90285426e-01 -4.56137914e-03  3.14972644e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [9]:
#!/usr/bin/python3

## UR5/UR10 Inverse Kinematics - Ryan Keating Johns Hopkins University

# ***** lib
import numpy as np
from numpy import linalg
import cmath
import math
from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi

global mat
mat = np.matrix

# ****** Coefficients ******
global d1, a2, a3, a7, d4, d5, d6
d1 = 0.1519
a2 = -0.24365
a3 = -0.21325
#a7 = 0.075
d4 = 0.11235
d5 = 0.08535
d6 = 0.0819

d = mat([0.1519, 0, 0, 0.11235, 0.08535, 0.0819])  # ur3 mm
a = mat([0, -0.612, -0.5723, 0, 0, 0])  # ur3 mm
alph = mat([pi / 2, 0, 0, pi / 2, -pi / 2, 0])  # ur3

In [10]:
# ************************************************** FORWARD KINEMATICS

def AH(n, th, c):

    T_a = mat(np.identity(4), copy=False)
    T_a[0, 3] = a[0, n - 1]
    T_d = mat(np.identity(4), copy=False)
    T_d[2, 3] = d[0, n - 1]

    Rzt = mat([ [cos(th[n - 1, c]), -sin(th[n - 1, c]), 0, 0],
                [sin(th[n - 1, c]), cos(th[n - 1, c]), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], copy=False)

    Rxa = mat([[1, 0, 0, 0],
                [0, cos(alph[0, n - 1]), -sin(alph[0, n - 1]), 0],
                [0, sin(alph[0, n - 1]), cos(alph[0, n - 1]), 0],
                [0, 0, 0, 1]], copy=False)

    A_i = T_d * Rzt * T_a * Rxa

    return A_i

def HTrans(th, c):
    A_1 = AH(1, th, c)
    A_2 = AH(2, th, c)
    A_3 = AH(3, th, c)
    A_4 = AH(4, th, c)
    A_5 = AH(5, th, c)
    A_6 = AH(6, th, c).90295797e-01 -1.38975612e-01  1.18989396e-04 -3.25740991e-01]
 [-5.16086992e-04 -4.53365110e-03 -9.99989590e-01 -2.23177803e-01]
 [ 1.38974705e-01  9.90285426e-01 -4.56137914e-03  3.14972644e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

    T_06 = A_1 * A_2 * A_3 * A_4 * A_5 * A_6

    return T_06

In [11]:
# ************************************************** INVERSE KINEMATICS

def invKine(desired_pos):  # T60
    th = mat(np.zeros((6, 8)))
    P_05 = (desired_pos * mat([0, 0, -d6, 1]).T - mat([0, 0, 0, 1]).T)

    # **** theta1 ****

    psi = atan2(P_05[2 - 1, 0], P_05[1 - 1, 0])
    phi = acos(d4 / sqrt(P_05[2 - 1, 0] * P_05[2 - 1, 0] + P_05[1 - 1, 0] * P_05[1 - 1, 0]))
    th[0, 0:4] = pi / 2 + psi + phi
    th[0, 4:8] = pi / 2 + psi - phi
    th = th.real

    # **** theta5 ****

    cl = [0, 4]  # wrist up or down
    for i in range(0, len(cl)):
        c = cl[i]
        T_10 = linalg.inv(AH(1, th, c))
        T_16 = T_10 * desired_pos
        th[4, c:c + 2] = +acos((T_16[2, 3] - d4) / d6)
        th[4, c + 2:c + 4] = -acos((T_16[2, 3] - d4) / d6)

    th = th.real

    # **** theta6 ****

    cl = [0, 2, 4, 6]
    for i in range(0, len(cl)):
        c = cl[i]
        T_10 = linalg.inv(AH(1, th, c))
        T_16 = linalg.inv(T_10 * desired_pos)
        th[5, c:c + 2] = atan2((-T_16[1, 2] / sin(th[4, c])), (T_16[0, 2] / sin(th[4, c])))

    th = th.real

    # **** theta3 ****
    cl = [0, 2, 4, 6]
    for i in range(0, len(cl)):
        c = cl[i]
        T_10 = linalg.inv(AH(1, th, c))
        T_65 = AH(6, th, c)
        T_54 = AH(5, th, c)
        T_14 = (T_10 * desired_pos) * linalg.inv(T_54 * T_65)
        P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0, 0, 0, 1]).T
        t3 = cmath.acos(
            (linalg.norm(P_13) ** 2 - a2 ** 2 - a3 ** 2) / (2 * a2 * a3))  # norm ?
        th[2, c] = t3.real
        th[2, c + 1] = -t3.real

    # **** theta2 and theta 4 ****

    cl = [0, 1, 2, 3, 4, 5, 6, 7]
    for i in range(0, len(cl)):
        c = cl[i]
        T_10 = linalg.inv(AH(1, th, c))
        T_65 = linalg.inv(AH(6, th, c))
        T_54 = linalg.inv(AH(5, th, c))
        T_14 = (T_10 * desired_pos) * T_65 * T_54
        P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0, 0, 0, 1]).T

        # theta 2
        th[1, c] = -atan2(P_13[1], -P_13[0]) + asin(a3 * sin(th[2, c]) / linalg.norm(P_13))
        # theta 4
        T_32 = linalg.inv(AH(3, th, c))
        T_21 = linalg.inv(AH(2, th, c))
        T_34 = T_32 * T_21 * T_14
        th[3, c] = atan2(T_34[1, 0], T_34[0, 0])
    th = th.real

    return th

In [12]:
# Import library yang diperlukan
import numpy as np

# Input posisi yang diinginkan (matriks transformasi T_06)
desired_pos = np.array(result_htm)

# Panggil fungsi inverse kinematics
result_ik = invKine(desired_pos)

# Tampilkan hasil
print("Solusi inverse kinematics (radian):")
print(result_ik)

Solusi inverse kinematics (radian):
[[ 0.08726646  0.08726646  0.08726646  0.08726646 -2.41042324 -2.41042324
  -2.41042324 -2.41042324]
 [-0.99483767 -0.26454525 -0.90059267  0.43834623 -3.57180765 -2.25493426
  -2.88726915 -2.12517837]
 [ 0.78539816 -0.78539816  1.45759944 -1.45759944  1.43258623 -1.43258623
   0.81990373 -0.81990373]
 [ 0.26179939  1.10230329  2.63694577 -2.06997957 -1.00920318  0.5390959
   2.06053348  2.93825016]
 [ 0.08726646  0.08726646 -0.08726646 -0.08726646  2.41053063  2.41053063
  -2.41053063 -2.41053063]
 [ 0.08726646  0.08726646 -3.05432619 -3.05432619 -3.00725137 -3.00725137
   0.13434128  0.13434128]]


In [13]:
result_ik_degrees = np.degrees(result_ik)

# Tampilkan hasil
print("Solusi inverse kinematics (derajat):")
print(result_ik_degrees)
print()


Solusi inverse kinematics (derajat):
[[   5.            5.            5.            5.         -138.10707867
  -138.10707867 -138.10707867 -138.10707867]
 [ -57.          -15.15732617  -51.60015912   25.11538882 -204.64950336
  -129.19821593 -165.42833646 -121.76375107]
 [  45.          -45.           83.51429589  -83.51429589   82.08114502
   -82.08114502   46.97702361  -46.97702361]
 [  15.           63.15732617  151.08586323 -118.60109293  -57.82308274
    30.88791986  118.05987177  168.3493336 ]
 [   5.            5.           -5.           -5.          138.1132317
   138.1132317  -138.1132317  -138.1132317 ]
 [   5.            5.         -175.         -175.         -172.30281167
  -172.30281167    7.69718833    7.69718833]]



In [14]:
result_ik_transposed = np.transpose(result_ik_degrees)

# Tampilkan hasil
print("Solusi inverse kinematics (dalam derajat):")
print(result_ik_transposed)

Solusi inverse kinematics (dalam derajat):
[[   5.          -57.           45.           15.            5.
     5.        ]
 [   5.          -15.15732617  -45.           63.15732617    5.
     5.        ]
 [   5.          -51.60015912   83.51429589  151.08586323   -5.
  -175.        ]
 [   5.           25.11538882  -83.51429589 -118.60109293   -5.
  -175.        ]
 [-138.10707867 -204.64950336   82.08114502  -57.82308274  138.1132317
  -172.30281167]
 [-138.10707867 -129.19821593  -82.08114502   30.88791986  138.1132317
  -172.30281167]
 [-138.10707867 -165.42833646   46.97702361  118.05987177 -138.1132317
     7.69718833]
 [-138.10707867 -121.76375107  -46.97702361  168.3493336  -138.1132317
     7.69718833]]


In [15]:
# Inisialisasi list untuk menyimpan pose kartesian
result_ik_transposed_list = result_ik_transposed.tolist()

In [16]:
print(result_ik_transposed_list)

[[4.999999999999981, -57.00000000000069, 45.000000000001094, 14.999999999999604, 4.999999999998897, 4.999999999999989], [4.999999999999981, -15.157326168067314, -45.000000000001094, 63.15732616806842, 4.999999999998897, 4.999999999999989], [4.999999999999981, -51.60015912188185, 83.51429589333685, 151.085863228545, -4.999999999998897, -175.0], [4.999999999999981, 25.115388821888196, -83.51429589333685, -118.60109292855135, -4.999999999998897, -175.0], [-138.1070786706421, -204.64950335822323, 82.08114501851468, -57.82308274186401, 138.11323169632954, -172.30281167169937], [-138.1070786706421, -129.1982159250459, -82.08114501851468, 30.887919861988028, 138.11323169632954, -172.30281167169937], [-138.1070786706421, -165.4283364647756, 46.97702361483047, 118.0598717683726, -138.11323169632954, 7.697188328300641], [-138.1070786706421, -121.76375107035938, -46.97702361483047, 168.34933360361728, -138.11323169632954, 7.697188328300641]]


In [18]:
#-------------------------------------------------------------------------------------------------
#print("\n")
#Result in Degree
# Result in Degree
print('8 Solusi IK (Derajat)')
print("[")
for sublist in result_ik_transposed_list:
    print("[", end=" ")
    for i, value in enumerate(sublist):
        # Print float values with higher precision
        if isinstance(value, float):
            if i == len(sublist) - 1:
                print("{:.15f}".format(value), end=" ")
            else:
                print("{:.15f}".format(value), end=", ")
        else:
            print(value, end=", ")
    print("],")  # Add a closing square bracket and comma for each sublist
print("]")

8 Solusi IK (Derajat)
[
[ 4.999999999999981, -57.000000000000689, 45.000000000001094, 14.999999999999604, 4.999999999998897, 4.999999999999989 ],
[ 4.999999999999981, -15.157326168067314, -45.000000000001094, 63.157326168068423, 4.999999999998897, 4.999999999999989 ],
[ 4.999999999999981, -51.600159121881852, 83.514295893336850, 151.085863228545008, -4.999999999998897, -175.000000000000000 ],
[ 4.999999999999981, 25.115388821888196, -83.514295893336850, -118.601092928551353, -4.999999999998897, -175.000000000000000 ],
[ -138.107078670642096, -204.649503358223228, 82.081145018514675, -57.823082741864013, 138.113231696329535, -172.302811671699374 ],
[ -138.107078670642096, -129.198215925045901, -82.081145018514675, 30.887919861988028, 138.113231696329535, -172.302811671699374 ],
[ -138.107078670642096, -165.428336464775612, 46.977023614830472, 118.059871768372602, -138.113231696329535, 7.697188328300641 ],
[ -138.107078670642096, -121.763751070359376, -46.977023614830472, 168.34933360361

In [22]:
d1 = 0.1519 
d2 = d3 = 0
d4 = 0.11235
d5 = 0.08535
d6 = 0.0819

# a (unit: mm)
a1 = a4 = a5 = a6 = 0
a2 = -0.24365
a3 = -0.21325

# List type of D-H parameter
d = np.array([d1, d2, d3, d4, d5, d6]) # unit: mm
a = np.array([a1, a2, a3, a4, a5, a6]) # unit: mm
alpha = np.array([pi/2, 0, 0, pi/2, -pi/2, 0]) # unit: radian

In [23]:
print("ROS Pose hasil forward kinematics dari nilai sendi hasil inverse kinematics:")
for index, solution in enumerate(result_ik_transposed_list, start=1):
    result_cartesian = fwd_kin(solution, i_unit='d', o_unit='p')
    print(f"Solution {index}:")
    print(result_cartesian)
    print()


ROS Pose hasil forward kinematics dari nilai sendi hasil inverse kinematics:
Solution 1:
position: 
  x: -0.32574099140030965
  y: -0.22317780274121768
  z: 0.31497264420402243
orientation: 
  x: 0.7069990853988244
  y: -0.04932527561612567
  z: 0.04918453819682154
  w: 0.703775668542885

Solution 2:
position: 
  x: -0.3257409914003096
  y: -0.22317780274121768
  z: 0.3149726442040225
orientation: 
  x: 0.7069990853988243
  y: -0.049325275616125694
  z: 0.04918453819682157
  w: 0.7037756685428849

Solution 3:
position: 
  x: -0.32574099140030965
  y: -0.22317780274121768
  z: 0.3149726442040225
orientation: 
  x: 0.7069990853988243
  y: -0.049325275616125694
  z: 0.049184538196821566
  w: 0.7037756685428849

Solution 4:
position: 
  x: -0.32574099140030965
  y: -0.22317780274121768
  z: 0.3149726442040225
orientation: 
  x: 0.7069990853988243
  y: -0.04932527561612563
  z: 0.04918453819682154
  w: 0.7037756685428849

Solution 5:
position: 
  x: -0.3257409914003136
  y: -0.2231778027412